In [1]:


import xarray as xr
import xesmf
import matplotlib.pyplot as plt
import bottleneck
import numpy as np
import subprocess as sp
import os
import glob
import cartopy.crs as ccrs



In [4]:


def open_grid(path,decode_times=False):
    """Return a grid object containing staggered grid locations"""
    grid={}
    grid['ds']=xr.open_dataset(path,decode_times=False)
    grid['ds']=grid['ds'].drop_dims(['ny','nx'])
    grid['ds']=grid['ds'].drop_vars(['tile'])
    grid['nyp']=grid['ds'].nyp.data[-1]+1
    grid['nxp']=grid['ds'].nxp.data[-1]+1
    nxp=grid['nxp'];nyp=grid['nyp']
    grid['h'] = grid['ds'].isel(nxp=slice(1,nxp+1,2),nyp=slice(1,nyp+1,2))
    #The q grid is not symmetric, but Cu and Cv are
    grid['q'] = grid['ds'].isel(nxp=slice(2,nxp+1,2),nyp=slice(2,nyp+1,2))
    grid['Cu'] = grid['ds'].isel(nxp=slice(0,nxp+1,2),nyp=slice(1,nyp+1,2))
    grid['Cv'] = grid['ds'].isel(nxp=slice(1,nxp+1,2),nyp=slice(0,nyp+1,2))
    return grid



In [5]:
r_grid = "/home/nma/mom/MOM6dev/exps/regional2/INPUT/bob_grid.nc"

region = open_grid(r_grid)
region

{'ds': <xarray.Dataset>
 Dimensions:   (nyp: 385, nxp: 525)
 Dimensions without coordinates: nyp, nxp
 Data variables:
     x         (nyp, nxp) float64 ...
     y         (nyp, nxp) float64 ...
     angle_dx  (nyp, nxp) float64 ...
     arcx      |S255 ...
 Attributes:
     grid_version:  0.2
     code_version:  $Name: fre-nctools-bronx-10 $
     history:       make_hgrid --grid_type regular_lonlat_grid --nxbnd 2 --nyb...,
 'nyp': 385,
 'nxp': 525,
 'h': <xarray.Dataset>
 Dimensions:   (nyp: 192, nxp: 262)
 Dimensions without coordinates: nyp, nxp
 Data variables:
     x         (nyp, nxp) float64 ...
     y         (nyp, nxp) float64 ...
     angle_dx  (nyp, nxp) float64 ...
     arcx      |S255 ...
 Attributes:
     grid_version:  0.2
     code_version:  $Name: fre-nctools-bronx-10 $
     history:       make_hgrid --grid_type regular_lonlat_grid --nxbnd 2 --nyb...,
 'q': <xarray.Dataset>
 Dimensions:   (nyp: 192, nxp: 262)
 Dimensions without coordinates: nyp, nxp
 Data variables:
 

In [ ]:
r_grid = "/home/nma/mom/MOM6dev/exps/regional2/INPUT/bob_grid.nc"

region = open_grid(r_grid)
region

{'ds': <xarray.Dataset>
 Dimensions:   (nyp: 385, nxp: 525)
 Dimensions without coordinates: nyp, nxp
 Data variables:
     x         (nyp, nxp) float64 ...
     y         (nyp, nxp) float64 ...
     angle_dx  (nyp, nxp) float64 ...
     arcx      |S255 ...
 Attributes:
     grid_version:  0.2
     code_version:  $Name: fre-nctools-bronx-10 $
     history:       make_hgrid --grid_type regular_lonlat_grid --nxbnd 2 --nyb...,
 'nyp': 385,
 'nxp': 525,
 'h': <xarray.Dataset>
 Dimensions:   (nyp: 192, nxp: 262)
 Dimensions without coordinates: nyp, nxp
 Data variables:
     x         (nyp, nxp) float64 ...
     y         (nyp, nxp) float64 ...
     angle_dx  (nyp, nxp) float64 ...
     arcx      |S255 ...
 Attributes:
     grid_version:  0.2
     code_version:  $Name: fre-nctools-bronx-10 $
     history:       make_hgrid --grid_type regular_lonlat_grid --nxbnd 2 --nyb...,
 'q': <xarray.Dataset>
 Dimensions:   (nyp: 192, nxp: 262)
 Dimensions without coordinates: nyp, nxp
 Data variables:
 

In [11]:
def open_dataset(ds,fields,grid):
    #ds=xr.open_dataset(path,decode_times=False)
    
    tracer_list=[];uv_list=[]
    for f in fields:
        for fnam,val in zip(f.keys(),f.values()):
            if val=='h':tracer_list.append(fnam)
            if val=='Cu':uv_list.append(fnam)
            if val=='Cv':uv_list.append(fnam)
                
    ds_tr = xr.merge([ds, grid['h']])
    ds_u= xr.merge([ds,grid['Cu']])
    ds_v= xr.merge([ds,grid['Cv']])
    return {'ds_tr':ds_tr,'ds_u':ds_u,'ds_v':ds_v,'tracers':tracer_list,'uv':uv_list}



In [19]:

IOmodel = "/home/nma/HDD/vinay/archives/IOMOM5Op/"

uu = xr.open_mfdataset(IOmodel+"*.ocean_uvel.nc")
vv = xr.open_mfdataset(IOmodel+"*.ocean_vvel.nc")
#fields=[{'temp':'h'},{'salt':'h'},{'ssh':'h'},{'u':'Cu'},{'v':'Cv'}]

fields=[{'u':'Cu'},{'v':'Cv'}]

dset = xr.merge([uu,vv])

final_open = open_dataset(dset,fields,region)



### Remap vel to q corners

In [45]:


def velocity_at_corners(ds_u,ds_v):
    nxp=ds_u.xu_ocean[-1].data+1;nyp=ds_v.xu_ocean[-1].data+1
    #upper-right q points
    u_q=0.5*(ds_u.u+ds_u.u.roll(roll_coords='yu_ocean',yu_ocean=-1)).isel(xu_ocean=slice(1,nxp))
    #upper-right q points
    v_q=0.5*(ds_v.v+ds_v.v.roll(roll_coords='xu_ocean',xu_ocean=-1)).isel(yu_ocean=slice(1,nyp))
    ds_uvq = xr.Dataset({'u':u_q,'v':v_q},coords={'time':ds_u.time,'lon':parent_grid['q'].x,'lat':parent_grid['q'].y,'angle_dx':parent_grid['q'].angle_dx})
    return ds_uvq



In [50]:
ds_u=final_open['ds_u'];ds_v=final_open['ds_v']

nxp=ds_u.xu_ocean[-1].data+1;nyp=ds_v.xu_ocean[-1].data+1
u_q=0.5*(ds_u.u+ds_u.u.roll(roll_coords='yu_ocean',yu_ocean=-1)).isel(xu_ocean=slice(1,nxp))
#final_open['ds_uv']=velocity_at_corners(ds_u.isel(time=slice(0,1)),ds_v.isel(time=slice(0,1)))

/home/nma/nma/miniconda3/envs/mom/lib/python3.8/site-packages/xarray/core/indexing.py:1227: PerformanceWarning: Slicing is producing a large chunk. To accept the large
chunk and silence this warning, set the option
    >>> with dask.config.set(**{'array.slicing.split_large_chunks': False}):
    ...     array[indexer]

To avoid creating the large chunks, set the option
    >>> with dask.config.set(**{'array.slicing.split_large_chunks': True}):
    ...     array[indexer]
  return self.array[key]


TypeError: 'numpy.float64' object cannot be interpreted as an integer